In [4]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sys, os

# Khởi tạo Dask client để tối ưu hiệu suất
client = Client()

file_path = "single_file.csv"
output_files = ["file1.csv", "file2.csv", "file3.csv"]
temp_dir = "FL_Data1/"  # Thư mục lưu file tạm
output_files = [temp_dir + output_file for output_file in output_files]
print(output_files)

# Cấu hình
chunksize = 100000  # Đọc từng phần (tuỳ vào RAM)

# Số lượng mẫu cần phân bổ cho từng file
target_samples = {
    output_files[0]: {'label_0': 463594, 'label_1': 410257},  # File 1
    output_files[1]: {'label_0': 338276, 'label_1': 2014056},  # File 2
    output_files[2]: {'label_0': 296325, 'label_1': 1575683}   # File 3
}

# Khởi tạo biến để theo dõi số lượng mẫu đã phân bổ
allocated_samples = {file: {'label_0': 0, 'label_1': 0} for file in output_files}

header_saved = False
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Tách dữ liệu theo nhãn (giả sử cột nhãn là 'label')
    label_0 = chunk[chunk['label'] == 0]
    label_1 = chunk[chunk['label'] == 1]

    # Tính số lượng mẫu còn lại cần phân bổ cho từng file
    df_chunks = {file: [] for file in output_files}
    for file in output_files:
        # Số lượng mẫu cần lấy từ chunk này
        remaining_0 = max(0, target_samples[file]['label_0'] - allocated_samples[file]['label_0'])
        remaining_1 = max(0, target_samples[file]['label_1'] - allocated_samples[file]['label_1'])

        # Số lượng mẫu thực tế có thể lấy từ chunk
        take_0 = min(remaining_0, len(label_0))
        take_1 = min(remaining_1, len(label_1))

        # Lấy mẫu ngẫu nhiên từ label_0 và label_1
        if take_0 > 0:
            sample_0 = label_0.sample(n=take_0, random_state=42)
            label_0 = label_0.drop(sample_0.index)
        else:
            sample_0 = pd.DataFrame(columns=chunk.columns)

        if take_1 > 0:
            sample_1 = label_1.sample(n=take_1, random_state=42)
            label_1 = label_1.drop(sample_1.index)
        else:
            sample_1 = pd.DataFrame(columns=chunk.columns)

        # Gộp mẫu label 0 và label 1
        df_chunk = pd.concat([sample_0, sample_1])
        df_chunks[file] = df_chunk

        # Cập nhật số lượng mẫu đã phân bổ
        allocated_samples[file]['label_0'] += take_0
        allocated_samples[file]['label_1'] += take_1

    # Lưu dữ liệu vào file
    for file in output_files:
        if not df_chunks[file].empty:
            df_chunks[file].to_csv(file, mode="a", header=not header_saved, index=False)

    header_saved = True

    # Kiểm tra nếu đã phân bổ đủ mẫu
    all_allocated = all(
        allocated_samples[file]['label_0'] >= target_samples[file]['label_0'] and
        allocated_samples[file]['label_1'] >= target_samples[file]['label_1']
        for file in output_files
    )
    if all_allocated:
        break

# Đóng client Dask
client.close()

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34929 instead
  warnings.warn(


['FL_Data1/file1.csv', 'FL_Data1/file2.csv', 'FL_Data1/file3.csv']


/tmp/ipykernel_7318/1306870874.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_chunk = pd.concat([sample_0, sample_1])
/tmp/ipykernel_7318/1306870874.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_chunk = pd.concat([sample_0, sample_1])
/tmp/ipykernel_7318/1306870874.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. 

In [2]:
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import pandas as pd
import sys, os

# Khởi tạo Dask client để tối ưu hiệu suất
client = Client()

file_path = "single_file.csv"
output_files = ["file1.csv", "file2.csv", "file3.csv"]
temp_dir = "FL_Data3/"  # Thư mục lưu file tạm
output_files = [temp_dir + output_file for output_file in output_files]
print(output_files)

# Định nghĩa tỉ lệ mong muốn cho từng file (label 0: label 1)
target_ratios = {
    output_files[0]: (463594, 410257),  # File 1: label 0: label 1 = 1:3
    output_files[1]: (338276, 2014056),  # File 2: label 0: label 1 = 1:5
    output_files[2]: (296325, 1575683)   # File 3: label 0: label 1 = 3:2
}

# Cấu hình
chunksize = 100000  # Đọc từng phần (tuỳ vào RAM)
header_saved = False

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Tách dữ liệu theo nhãn (giả sử cột nhãn là 'label')
    label_0 = chunk[chunk['label'] == 0]
    label_1 = chunk[chunk['label'] == 1]

    # Tính tổng số mẫu trong chunk
    total_samples = len(chunk)
    total_label_0 = len(label_0)
    total_label_1 = len(label_1)

    if total_label_0 == 0 or total_label_1 == 0:
        print(f"Chunk {chunk.index[0]} không có đủ nhãn để chia!")
        continue

    # Tính số lượng mẫu cần lấy cho từng file dựa trên tỉ lệ
    file_samples = {file: {'label_0': 0, 'label_1': 0} for file in output_files}

    # Phân bổ mẫu theo tỉ lệ
    remaining_label_0 = total_label_0
    remaining_label_1 = total_label_1

    for file, (ratio_0, ratio_1) in target_ratios.items():
        # Tính tỷ lệ tổng (ratio_0 + ratio_1)
        total_ratio = ratio_0 + ratio_1
        # Ước lượng số mẫu cho file này dựa trên tỉ lệ tổng
        file_size = int(total_samples / len(output_files))  # Phân bổ đều ban đầu
        label_0_count = int(file_size * (ratio_0 / total_ratio))
        label_1_count = int(file_size * (ratio_1 / total_ratio))

        # Điều chỉnh để không vượt quá số lượng mẫu còn lại
        label_0_count = min(label_0_count, remaining_label_0)
        label_1_count = min(label_1_count, remaining_label_1)

        file_samples[file]['label_0'] = label_0_count
        file_samples[file]['label_1'] = label_1_count
        remaining_label_0 -= label_0_count
        remaining_label_1 -= label_1_count

    # Phân bổ phần còn lại (nếu có) để đảm bảo không bỏ sót mẫu
    for file in output_files:
        if remaining_label_0 > 0:
            file_samples[file]['label_0'] += 1
            remaining_label_0 -= 1
        if remaining_label_1 > 0:
            file_samples[file]['label_1'] += 1
            remaining_label_1 -= 1

    # Lấy mẫu ngẫu nhiên từ từng nhóm nhãn
    df_1 = pd.concat([label_0.sample(n=file_samples[output_files[0]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[0]]['label_1'], random_state=42)])
    df_2 = pd.concat([label_0.sample(n=file_samples[output_files[1]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[1]]['label_1'], random_state=42)])
    df_3 = pd.concat([label_0.sample(n=file_samples[output_files[2]]['label_0'], random_state=42),
                      label_1.sample(n=file_samples[output_files[2]]['label_1'], random_state=42)])

    # Lưu dữ liệu vào file
    df_1.to_csv(output_files[0], mode="a", header=not header_saved, index=False)
    df_2.to_csv(output_files[1], mode="a", header=not header_saved, index=False)
    df_3.to_csv(output_files[2], mode="a", header=not header_saved, index=False)
    header_saved = True

# Đóng client Dask
client.close()

/home/th/miniconda3/envs/tf/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35693 instead
  warnings.warn(


['FL_Data3/file1.csv', 'FL_Data3/file2.csv', 'FL_Data3/file3.csv']
